In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 

In [2]:
input_green='data/pq/green/*/*'
input_yellow='data/pq/yellow/*/*'
output='data/report/revenue/'

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 23:06:12 WARN Utils: Your hostname, codespaces-7c27fa resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/02 23:06:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 23:06:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# df_green = spark.read.format("parquet").load('data/pq/green/2020/08')
# # df_green = spark.read.format("parquet").option("recursiveFileLookup","true").load("data/raw/green/2021/*")
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.show()

In [ ]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_green.show()

In [ ]:
df_yellow = spark.read.parquet(input_yellow)


df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_yellow.show()

In [ ]:
common_colums = [
    'VendorID',
    'pickup_datetime',
    'dropoff_datetime',
    'store_and_fwd_flag',
    'RatecodeID',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'payment_type',
    'congestion_surcharge'
]


In [ ]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [ ]:
df_green_sel.show()

In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [ ]:
df_trips_data.count()

In [ ]:
df_trips_data.registerTempTable('trips_data')

In [ ]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

In [ ]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [ ]:
df_result.show()

In [ ]:
df_result.coalesce(1) \
    .write.parquet(output, mode='overwrite')

In [4]:
spark.stop()